<a href="https://colab.research.google.com/github/theorska/Homework3/blob/master/Weatherpred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Importing of the function that I am going to use
import numpy as np
from sklearn import preprocessing
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import SGD

In [41]:
#This is where the csv is stored. It is already in order, so there is no need for sorting.
url = 'https://raw.githubusercontent.com/theorska/Homework3/master/Munkaf%C3%BCzet.csv'
dataset_raw = np.genfromtxt(url,  delimiter=',' )
#Dut ot exporting, there are a few nan colums, lets get rid of them
dataset=dataset_raw[: , 1:3]
print("This is the dataset used for the training, validation")
print(dataset)

This is the dataset used for the training, validation
[[ 18.    6.4]
 [ 14.3   6.8]
 [ 16.8   3.5]
 [ 15.7   5.7]
 [ 17.1   2.9]
 [ 20.3   4. ]
 [ 20.3  12.7]
 [ 19.6  11.9]
 [ 22.7  10. ]
 [ 23.1   9.2]
 [ 22.3   9. ]
 [ 21.    8.6]
 [ 21.4   8.2]
 [ 19.5   8.3]
 [ 19.4   7.5]
 [ 18.8   8.5]
 [ 21.2   8.8]
 [ 21.1   9.4]
 [ 21.    8.9]
 [ 19.6   7.2]
 [ 10.8   8.3]
 [ 15.    7.7]
 [ 12.7   8.2]
 [ 12.8   6.8]
 [ 14.3   6. ]
 [ 14.1   7.1]
 [ 13.9  10.6]
 [ 20.7  11.4]
 [ 20.4  16. ]
 [ 18.1  10.8]
 [ 18.4   9.9]
 [ 20.8   8.8]
 [ 21.2  13. ]
 [ 19.6  11.3]
 [ 17.3   7.9]
 [ 17.2   6.6]
 [ 15.8   7.3]
 [ 15.6   6.6]
 [ 14.3   5.4]
 [ 12.7   3. ]
 [ 10.9   3.2]
 [ 15.    4.8]
 [ 14.7   4.3]
 [ 13.8   5.8]
 [ 12.    3.5]
 [ 10.7   0.6]
 [  8.3   0.9]
 [  5.9  -1.5]
 [  6.4  -3.6]
 [  1.9  -0.5]
 [  4.5   0.3]
 [  4.3   3.4]
 [  4.9   2.4]
 [  4.8   2.7]
 [  6.4   1.5]
 [  6.9   4.7]
 [  9.4   5.3]
 [  4.6   0.9]
 [  1.6  -3.8]
 [ -0.2  -6.3]
 [ -2.1  -5.3]
 [ -1.8  -5.6]
 [  0.3  -4.4]
 

In [42]:
# Here I save a few useful variables
number_of_days=30; #How many days we are using for the perdiction
week=7; #Number of days in a week
month=30; # Number of days in a month
size=dataset.shape[0]-month; #Size of the staset, thats useful, since if I want to predict for a month, I can't predict from the last ones, as I don't have the corrseponding data for the validation

#This is where I determine the sizes
number_of_test_data=int(size*0.9)
number_of_valid_data=size-number_of_test_data
train_input=np.zeros([number_of_test_data, number_of_days,dataset.shape[1] ] )
train_output=np.zeros([number_of_test_data, 3])
valid_input=np.zeros([number_of_valid_data, number_of_days,dataset.shape[1] ] )
valid_output=np.zeros([number_of_valid_data, 3])
print("Shape of the training input")
print(train_input.shape)
print("Shape of the training output")
print(train_output.shape)
print("Shape of the training ouput")
print(valid_input.shape)
print("Shape of the training ouput")
print(valid_output.shape)


#We have to fill them with the data, so for each member we take the 30 days before it and the avarage of the day, week, month we are looking for
for i in range(number_of_test_data):
  for j in range(number_of_days):
    train_input[i, j, 0]=dataset[i-j-1, 0]
    train_input[i, j, 1]=dataset[i-j-1, 1]
  train_output[i, 0]=(dataset[i-j, 0]-dataset[i-j, 1])/2
  train_output[i, 1]=(dataset[i-j+week, 0]-dataset[i-j+week, 1])/2
  train_output[i, 2]=(dataset[i-j+month, 0]-dataset[i-j+month, 1])/2

for i in range(number_of_valid_data):
  for j in range(number_of_days):
    valid_input[i, j, 0]=dataset[number_of_test_data+i-j-1, 0]
    valid_input[i, j, 1]=dataset[number_of_test_data+i-j-1, 1]
  valid_output[i, 0]=(dataset[number_of_test_data+i-j, 0]-dataset[number_of_test_data+i-j, 1])/2
  valid_output[i, 1]=(dataset[number_of_test_data+i-j+week, 0]-dataset[number_of_test_data+i-j+week, 1])/2
  valid_output[i, 2]=(dataset[number_of_test_data+i-j+month, 0]-dataset[number_of_test_data+i-j+month, 1])/2

#We need to reshape them for the input of the model   
train_input=train_input.reshape(number_of_test_data, -1)
valid_input=valid_input.reshape(number_of_valid_data, -1)
print("Reshaped values of the inputs")
print(train_input.shape)
print(valid_input.shape)

#Now its time to scale them
scaler = preprocessing.StandardScaler().fit(dataset[0:,1:2])
train_input=scaler.transform(train_input)
valid_input=scaler.transform(valid_input)

Shape of the training input
(326, 30, 2)
Shape of the training output
(326, 3)
Shape of the training ouput
(37, 30, 2)
Shape of the training ouput
(37, 3)
Reshaped values of the inputs
(326, 60)
(37, 60)


In [43]:
#Time to create the model
# Create the model
model = Sequential()

# The Input Layer :
model.add(Dense(64, kernel_initializer='normal',input_dim = number_of_days*2, activation='relu'))

# The Hidden Layers :
model.add(Dense(128, kernel_initializer='normal',activation='relu'))

# The Output Layer : 3 is the output as we are predicting for 3 days
model.add(Dense(3, kernel_initializer='normal',activation='linear'))

sgd = SGD(lr=1e-3, decay=1e-6,momentum=0.9, nesterov=True)
model.compile(loss='mse',optimizer=sgd)

# Compile the network :
model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 64)                3904      
_________________________________________________________________
dense_4 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 387       
Total params: 12,611
Trainable params: 12,611
Non-trainable params: 0
_________________________________________________________________


In [0]:
# Lets set early stopping and save the best model
patience=100
early_stopping= EarlyStopping(patience=patience, verbose=1)
checkpointer=ModelCheckpoint(filepath="weights.hdf5",save_best_only=True, verbose=1)

In [45]:
#Lets start the training
history=model.fit(train_input, train_output, epochs=1000,
                  batch_size=16,
                  verbose=2,
                  validation_data=(valid_input, valid_output),
                  callbacks=[checkpointer,early_stopping])

Train on 326 samples, validate on 37 samples
Epoch 1/1000

Epoch 00001: val_loss improved from inf to 0.82577, saving model to weights.hdf5
326/326 - 0s - loss: 3.2313 - mean_absolute_error: 3.2313 - val_loss: 0.8258 - val_mean_absolute_error: 0.8258
Epoch 2/1000

Epoch 00002: val_loss did not improve from 0.82577
326/326 - 0s - loss: 1.7633 - mean_absolute_error: 1.7633 - val_loss: 0.8999 - val_mean_absolute_error: 0.8999
Epoch 3/1000

Epoch 00003: val_loss improved from 0.82577 to 0.72215, saving model to weights.hdf5
326/326 - 0s - loss: 1.2799 - mean_absolute_error: 1.2799 - val_loss: 0.7222 - val_mean_absolute_error: 0.7222
Epoch 4/1000

Epoch 00004: val_loss improved from 0.72215 to 0.72196, saving model to weights.hdf5
326/326 - 0s - loss: 1.0958 - mean_absolute_error: 1.0958 - val_loss: 0.7220 - val_mean_absolute_error: 0.7220
Epoch 5/1000

Epoch 00005: val_loss improved from 0.72196 to 0.64577, saving model to weights.hdf5
326/326 - 0s - loss: 1.0542 - mean_absolute_error: 1.0

In [0]:
#Lets create a test data for the accroding dates, the last tdate in the datset in oct.29 so we start from there
test_input=np.zeros((1, 60))
for i in range(30):
  test_input[0 ,i*2]=dataset[dataset.shape[0]-1-i, 0]
  test_input[0, i*2+1]=dataset[dataset.shape[0]-1-i, 1]
test_input=scaler.transform(test_input)

In [51]:
#Lets load the model and see the results
from tensorflow.keras.models import load_model

model = load_model('weights.hdf5')

preds=model.predict(test_input)
print("Here are the predictions")
print("10.30:")
print(preds[0 , 0])
print("11.5:")
print(preds[0 , 1])
print("11.26:")
print(preds[0 , 2])

Here are the predictions
10.30:
4.0689354
11.5:
2.015137
11.26:
4.140536
